# Backend Target V2 API - Complete Guide for Qiskit Certification

**Section 4: Run Circuits - Part 3**

> **Exam Weight**: Part of 15% | **Must Master**: ✅✅

This notebook covers the Backend V2 API - the modern interface for accessing quantum hardware information!

**📌 Note**: This notebook uses `FakeBackend` from `qiskit_ibm_runtime.fake_provider` for testing without real hardware. The V2 API is identical for both fake and real backends!

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- Understand V2 vs V1 API differences
- Use `backend.target` and `operation_names`
- Check instruction support with `instruction_supported()`
- Access qubit properties and coupling maps
- Identify deprecated V1 methods
- Use FakeBackend for local testing

---

## 💡 Conceptual Deep Dive: The Library Card Analogy

**Backend Target = Library Catalog System**

| Library System | Backend Target |
|----------------|----------------|
| **Old card catalog** | V1 API (scattered methods) |
| **Digital database** | V2 API (unified Target) |
| **Book availability** | `instruction_supported()` |
| **Book details** | Gate properties (error, duration) |

```
V1 API (Legacy, Pre-2023):          V2 API (Current, 2023+):
┌──────────────────────────┐       ┌──────────────────────────┐
│ Backend                  │       │ Backend V2               │
│  ├─ configuration()      │       │  └─ target              │
│  ├─ properties()         │       │      ├─ operations       │
│  ├─ defaults()           │       │      ├─ qargs           │
│  └─ coupling_map         │       │      ├─ properties      │
└──────────────────────────┘       │      └─ instructions    │
Multiple scattered methods         └──────────────────────────┘
                                   Single unified interface
```

**Why V2 Matters**: V2 is the standard for Qiskit 2.X certification!

---

## Setup

Import required libraries:

In [ ]:
# Setup - Using FakeBackend for local testing
from qiskit import QuantumCircuit
from qiskit_ibm_runtime.fake_provider import FakeManilaV2, FakeSherbrooke

# Use FakeBackend for testing without real hardware
backend = FakeManilaV2()  # 5-qubit fake backend

print("=" * 60)
print("Backend API Versions Comparison")
print("=" * 60)

# V1 Backend (OLD - DEPRECATED):
#   backend.configuration()      # Returns BackendConfiguration
#   backend.properties()         # Returns BackendProperties
#   config.basis_gates           # List of supported gates

# V2 Backend (NEW - STANDARD):
target = backend.target                    # Returns Target object
ops = backend.target.operation_names       # Replaces basis_gates

print(f"Target object: {type(target)}")
print(f"Operation names: {ops}")

print("\n🎯 EXAM TIP: V2 is the standard in Qiskit 2.X!")
print("✓ Use backend.target not configuration()")
print("✓ Use operation_names not basis_gates")
print(f"\n✅ Using FakeBackend: {backend.name}")

## Part 1: V2 vs V1 API Comparison

### ⚠️ EXAM TRAP: V2 API Changes

```python
# V1 (DEPRECATED)
config = backend.configuration()  # ❌
gates = config.basis_gates         # ❌

# V2 (CORRECT)
target = backend.target            # ✓
gates = target.operation_names     # ✓
```

## Part 2: Backend Target Basics

In [ ]:
# Backend Target Basics - Using FakeBackend
from qiskit_ibm_runtime.fake_provider import FakeManilaV2

backend = FakeManilaV2()

# Get target from backend
target = backend.target

# Get supported operations
ops = target.operation_names
print("Supported Operations:")
print(ops)

# Number of qubits
num_qubits = target.num_qubits
print(f"\nNumber of qubits: {num_qubits}")

print("\n" + "=" * 50)
print("✓ target is an attribute (not a method!)")
print("✓ operation_names includes measure, reset, etc.")
print("✓ More detailed than V1 basis_gates")

## Part 3: Operation Names vs Basis Gates

In [ ]:
# Operation Names Comparison
from qiskit_ibm_runtime.fake_provider import FakeManilaV2

backend = FakeManilaV2()
target = backend.target

# V2 operation_names (all operations)
all_ops = target.operation_names
print("V2 operation_names (all operations):")
print(all_ops)

# Separate gates from non-gate operations
gate_ops = [op for op in all_ops if op not in ['measure', 'reset', 'delay']]
non_gate_ops = [op for op in all_ops if op in ['measure', 'reset', 'delay']]

print(f"\nGate operations: {gate_ops}")
print(f"Non-gate operations: {non_gate_ops}")

print("\n" + "=" * 50)
print("Key Differences:")
print("  V1 basis_gates: Only gate operations")
print("  V2 operation_names: Gates + measure + reset + delay")
print("\n✓ V2 operation_names is more comprehensive")

### Common IBM Operation Names

| Operation | Type | Description |
|-----------|------|-------------|
| `cx` | Gate | CNOT (default entangling gate) |
| `ecr` | Gate | Echoed Cross-Resonance (some backends) |
| `rz` | Gate | Z-rotation (native parameterized) |
| `sx` | Gate | √X gate |
| `x` | Gate | Pauli X |
| `id` | Gate | Identity |
| `measure` | Operation | Measurement |
| `reset` | Operation | Qubit reset to |0⟩ |
| `delay` | Operation | Wait time (for timing) |

### V1 → V2 Translation Table (MEMORIZE!)

| **Feature** | **V1 API (Old)** | **V2 API (New)** |
|-------------|------------------|------------------|
| Get basis gates | `backend.configuration().basis_gates` | `backend.target.operation_names` |
| Get coupling map | `backend.configuration().coupling_map` | `backend.target.build_coupling_map()` |
| Check gate support | Manual parsing | `target.instruction_supported('cx', (0,1))` |
| Gate error rate | `backend.properties().gate_error(...)` | `target['cx'][(0,1)].error` |
| Number of qubits | `backend.configuration().n_qubits` | `backend.num_qubits` |

## Part 4: instruction_supported() (EXAM CRITICAL!)

In [ ]:
# Check instruction support - EXAM CRITICAL!
from qiskit_ibm_runtime.fake_provider import FakeManilaV2

backend = FakeManilaV2()
target = backend.target

print("instruction_supported() Examples:")
print("=" * 50)

# Check single-qubit gate on qubit 0
sx_supported = target.instruction_supported('sx', qargs=(0,))
print(f"SX gate on qubit 0: {sx_supported}")

# Check single-qubit gate on qubit 2
x_supported = target.instruction_supported('x', qargs=(2,))
print(f"X gate on qubit 2: {x_supported}")

# Check two-qubit gate (depends on coupling map!)
cx_01 = target.instruction_supported('cx', qargs=(0, 1))
cx_02 = target.instruction_supported('cx', qargs=(0, 2))
cx_13 = target.instruction_supported('cx', qargs=(1, 3))

print(f"\nCX gate on qubits (0,1): {cx_01}")
print(f"CX gate on qubits (0,2): {cx_02}")
print(f"CX gate on qubits (1,3): {cx_13}")

# Show coupling map
coupling_map = target.build_coupling_map()
print(f"\nCoupling Map: {coupling_map.get_edges()}")

print("\n" + "=" * 50)
print("✓ qargs is tuple of qubit indices")
print("✓ Returns True/False for gate availability")
print("✓ Respects coupling map constraints")

### ⚠️ EXAM TRAP: qargs Format

```python
# CORRECT - tuple
target.instruction_supported('cx', qargs=(0, 1))  # ✓
target.instruction_supported('sx', qargs=(0,))    # ✓

# WRONG - list or no comma for single qubit
target.instruction_supported('cx', qargs=[0, 1])  # ❌
target.instruction_supported('sx', qargs=(0))     # ❌
```

## Part 5: Target Properties

In [ ]:
# Target Properties - Accessing Gate Details
from qiskit_ibm_runtime.fake_provider import FakeManilaV2

backend = FakeManilaV2()
target = backend.target

print("Target Properties:")
print("=" * 50)

# Number of qubits
print(f"Number of qubits: {target.num_qubits}")

# All operations
print(f"All operations: {target.operation_names}")

# Get gate properties (error rates, duration)
print("\n--- Gate Properties ---")

# CX gate properties on qubits (0, 1)
if target.instruction_supported('cx', qargs=(0, 1)):
    cx_props = target['cx'][(0, 1)]
    print(f"\nCX on (0,1):")
    print(f"  Error rate: {cx_props.error:.6f}" if cx_props.error else "  Error rate: N/A")
    print(f"  Duration: {cx_props.duration:.2e} s" if cx_props.duration else "  Duration: N/A")

# SX gate properties on qubit 0
if target.instruction_supported('sx', qargs=(0,)):
    sx_props = target['sx'][(0,)]
    print(f"\nSX on (0,):")
    print(f"  Error rate: {sx_props.error:.6f}" if sx_props.error else "  Error rate: N/A")
    print(f"  Duration: {sx_props.duration:.2e} s" if sx_props.duration else "  Duration: N/A")

print("\n" + "=" * 50)
print("Target provides:")
print("  - Per-gate properties")
print("  - Per-qubit properties")
print("  - Coupling map information")
print("  - Error rates and durations")

## Part 6: V2 with Transpilation

In [ ]:
# Target Integration with Transpilation
from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime.fake_provider import FakeManilaV2

backend = FakeManilaV2()

# Create a simple circuit
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

print("Original Circuit:")
print(qc.draw())

# V2 backend automatically uses target
transpiled = transpile(qc, backend=backend)

print("\nTranspiled Circuit (uses backend.target):")
print(transpiled.draw())

# Can also pass target explicitly
transpiled_explicit = transpile(qc, target=backend.target)

print("\nTranspiled with explicit target:")
print(f"Depth: {transpiled_explicit.depth()}")
print(f"Gates: {transpiled_explicit.count_ops()}")

print("\n" + "=" * 50)
print("✓ transpile() auto-extracts from V2 backend")
print("✓ No need to manually specify constraints")
print("✓ Target contains all necessary information")

## Part 7: Using Real Quantum Hardware (Reference)

When ready to use real IBM Quantum hardware, use `QiskitRuntimeService`:

```python
# For Real Hardware (requires IBM Quantum account)
from qiskit_ibm_runtime import QiskitRuntimeService

# Initialize service (first time requires token)
service = QiskitRuntimeService()  # Uses saved credentials
# OR: service = QiskitRuntimeService(channel="ibm_quantum", token="YOUR_TOKEN")

# Get a real backend
real_backend = service.backend("ibm_brisbane")  # 127-qubit backend

# Access target (same V2 API!)
target = real_backend.target
print(target.operation_names)
print(target.num_qubits)
```

**Note**: FakeBackend and real backends use the **same V2 API**!

In [ ]:
# Comparison: FakeBackend vs Real Backend (using FakeSherbrooke as example)
from qiskit_ibm_runtime.fake_provider import FakeManilaV2, FakeSherbrooke

print("=" * 60)
print("FakeBackend Comparison")
print("=" * 60)

# Small backend (5 qubits)
small_backend = FakeManilaV2()
print(f"\nFakeManilaV2:")
print(f"  Qubits: {small_backend.target.num_qubits}")
print(f"  Operations: {small_backend.target.operation_names}")

# Large backend (127 qubits - same as ibm_sherbrooke)
large_backend = FakeSherbrooke()
print(f"\nFakeSherbrooke (127 qubits):")
print(f"  Qubits: {large_backend.target.num_qubits}")
print(f"  Operations: {large_backend.target.operation_names}")

print("\n" + "=" * 60)
print("Available FakeBackends in qiskit_ibm_runtime.fake_provider:")
print("  - FakeManilaV2 (5 qubits)")
print("  - FakeKyoto (127 qubits)")
print("  - FakeSherbrooke (127 qubits)")
print("  - FakeAlmadenV2, FakeAthensV2, etc.")
print("\n✓ Use FakeBackends for testing without real hardware!")
print("✓ Same V2 API as real backends!")

## 📝 Practice Questions

### Question 1: V2 API Access

**How to get supported operations in V2?**

A) `backend.configuration().basis_gates`  
B) `backend.target.operation_names`  
C) `backend.basis_gates`  
D) `backend.operations()`

<details>
<summary>Answer</summary>

**B) `backend.target.operation_names`**

```python
# V2 (CORRECT)
ops = backend.target.operation_names  # ✓

# V1 (DEPRECATED)
config = backend.configuration()      # ❌
gates = config.basis_gates            # ❌
```
</details>

---

### Question 2: Checking Gate Support

**How to check if CX supported on qubits (0,1)?**

A) `backend.target.has_gate('cx', (0,1))`  
B) `backend.target.instruction_supported('cx', qargs=(0,1))`  
C) `backend.supports('cx', [0,1])`  
D) `backend.can_execute('cx', 0, 1)`

<details>
<summary>Answer</summary>

**B) `backend.target.instruction_supported('cx', qargs=(0,1))`**

```python
supported = backend.target.instruction_supported(
    'cx', 
    qargs=(0, 1)  # Tuple!
)
```

**Note**: qargs must be tuple, not list!
</details>

---

### Question 3: V1 vs V2

**Which is V2 standard?**

A) `backend.configuration()`  
B) `backend.properties()`  
C) `backend.target`  
D) `backend.setup()`

<details>
<summary>Answer</summary>

**C) `backend.target`**

V2 uses Target object:
- `backend.target` (attribute, not method)
- `backend.target.operation_names`
- `backend.target.instruction_supported()`

V1 deprecated:
- `backend.configuration()`
- `backend.properties()`
</details>

---

### Question 4: Operation Names

**What does operation_names include that basis_gates doesn't?**

A) Nothing - they're identical  
B) measure, reset, delay  
C) Only measurement  
D) Custom gates

<details>
<summary>Answer</summary>

**B) measure, reset, delay**

```python
# V1 basis_gates (gates only)
['cx', 'rz', 'sx', 'x', 'id']

# V2 operation_names (all operations)
['cx', 'rz', 'sx', 'x', 'id', 'measure', 'reset', 'delay']
```

V2 includes non-gate operations!
</details>

---

## ✅ Key Takeaways

### Core Concepts

1. **V2 API (Standard)**
   - `backend.target` (attribute)
   - `backend.target.operation_names`
   - `backend.target.instruction_supported()`

2. **V1 API (Deprecated)**
   - `backend.configuration()`
   - `backend.properties()`
   - `config.basis_gates`

3. **Key Differences**
   - operation_names > basis_gates (includes measure, reset)
   - Target: per-qubit, per-gate properties
   - instruction_supported() checks specific qubits

### Critical Exam Facts

- ✅ V2 is Qiskit 2.X standard
- ✅ `backend.target` not `backend.configuration()`
- ✅ `operation_names` not `basis_gates`
- ✅ `instruction_supported(op, qargs=(i,j))`
- ✅ qargs must be tuple: `(0,)` not `(0)`
- ✅ operation_names includes measure/reset

### Common Traps

- ❌ `backend.configuration()` → ✅ `backend.target`
- ❌ `config.basis_gates` → ✅ `target.operation_names`
- ❌ `qargs=[0,1]` → ✅ `qargs=(0,1)`
- ❌ `qargs=(0)` → ✅ `qargs=(0,)`

### Mnemonic

🧠 **"V2 Target, not Configuration!"**

**Section 4 Complete!** ✅